<a href="https://colab.research.google.com/github/naveen-pixel07/2025_ML/blob/main/Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=0, n_repeated=0, n_classes=3, n_clusters_per_class=1, weights=[0.5, 0.3, 0.2], flip_y=0.01, random_state=42)

In [11]:
import numpy as np
from scipy.stats import multivariate_normal

def calculate_prior(y_train):
    classes, counts = np.unique(y_train, return_counts=True)
    prior_prob = counts / len(y_train)
    return dict(zip(classes, prior_prob))

def calculate_likelihood(X_train, y_train, x):
    classes = np.unique(y_train)
    likelihoods = {}
    for c in classes:
        X_c = X_train[y_train == c]
        mean_c = np.mean(X_c, axis=0)
        covariance_c = np.cov(X_c.T) + np.eye(X_c.shape[1]) * 1e-6
        likelihoods[c] = multivariate_normal.pdf(x, mean=mean_c, cov=covariance_c)
    return likelihoods

def calculate_posterior(prior_prob, likelihoods):
    posterior = {}
    evidence = sum(prior_prob[c] * likelihoods[c] for c in prior_prob)
    for c in prior_prob:
        posterior[c] = (prior_prob[c] * likelihoods[c]) / (evidence if evidence != 0 else 1e-9)
    return posterior

def classify(X_train, y_train, x):
    prior_prob = calculate_prior(y_train)
    likelihoods = calculate_likelihood(X_train, y_train, x)
    posterior = calculate_posterior(prior_prob, likelihoods)
    return max(posterior, key=posterior.get)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

predicted_labels = []

for test_point in X_test:
    predicted_class = classify(X_train, y_train, test_point)
    predicted_labels.append(predicted_class)

predicted_labels = np.array(predicted_labels)

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Overall Accuracy: {accuracy:.4f}")

weighted_precision = precision_score(y_test, predicted_labels, average='weighted')
weighted_recall = recall_score(y_test, predicted_labels, average='weighted')
weighted_f1 = f1_score(y_test, predicted_labels, average='weighted')

print(f"Overall Precision (weighted): {weighted_precision:.4f}")
print(f"Overall Recall (weighted): {weighted_recall:.4f}")
print(f"Overall F1-Score (weighted): {weighted_f1:.4f}")

Overall Accuracy: 0.9500
Overall Precision (weighted): 0.9509
Overall Recall (weighted): 0.9500
Overall F1-Score (weighted): 0.9500
